# Bureau of Labor Statistics
https://www.bls.gov/oes/about_charts.htm

## EDA

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_excel('/content/MSA_M2019_dl.xlsx')

In [4]:
print(df.shape)
df.head()

(142410, 30)


,area,area_title,area_type,naics,naics_title,i_group,own_code,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000,loc_quotient,pct_total,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,annual,hourly
0,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,00-0000,All Occupations,total,66510,3.1,1000,1,NaN,20.08,41760,1.8,8.94,10.97,15.64,24.05,35.21,18580,22810,32530,50030,73230,NaN,NaN
1,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,11-0000,Management Occupations,major,2670,5.8,40.137,0.73,NaN,43.51,90500,2.2,17.86,27.22,37.51,51.88,75.57,37140,56610,78030,107910,157190,NaN,NaN
2,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,11-1021,General and Operations Managers,detailed,1100,7.7,16.499,1.01,NaN,40.39,84000,3.7,10.97,24.21,35.51,51.38,74.29,22810,50350,73850,106870,154520,NaN,NaN
3,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,11-2022,Sales Managers,detailed,110,18.8,1.728,0.63,NaN,59.96,124710,6.6,22.77,34.69,57.65,74.55,99.8,47360,72140,119920,155050,207570,NaN,NaN
4,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,11-2030,Public Relations and Fundraising Managers,detailed,40,29.7,0.571,1.06,NaN,*,*,*,*,*,*,*,*,*,*,*,*,*,NaN,NaN


In [5]:
df.dtypes

area              int64
area_title       object
area_type         int64
naics             int64
naics_title      object
i_group          object
own_code          int64
occ_code         object
occ_title        object
o_group          object
tot_emp          object
emp_prse         object
jobs_1000        object
loc_quotient     object
pct_total       float64
h_mean           object
a_mean           object
mean_prse        object
h_pct10          object
h_pct25          object
h_median         object
h_pct75          object
h_pct90          object
a_pct10          object
a_pct25          object
a_median         object
a_pct75          object
a_pct90          object
annual           object
hourly           object
dtype: object

## Data Cleaning

In [6]:
# Dropping pct_total, annual, and hourly
# Majority is empty
bls_m2019 = df.copy()
bls_m2019 = bls_m2019.dropna(axis=1)

In [7]:
print(bls_m2019.shape)
bls_m2019.head()

(142410, 27)


,area,area_title,area_type,naics,naics_title,i_group,own_code,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90
0,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,00-0000,All Occupations,total,66510,3.1,1000,1,20.08,41760,1.8,8.94,10.97,15.64,24.05,35.21,18580,22810,32530,50030,73230
1,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,11-0000,Management Occupations,major,2670,5.8,40.137,0.73,43.51,90500,2.2,17.86,27.22,37.51,51.88,75.57,37140,56610,78030,107910,157190
2,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,11-1021,General and Operations Managers,detailed,1100,7.7,16.499,1.01,40.39,84000,3.7,10.97,24.21,35.51,51.38,74.29,22810,50350,73850,106870,154520
3,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,11-2022,Sales Managers,detailed,110,18.8,1.728,0.63,59.96,124710,6.6,22.77,34.69,57.65,74.55,99.8,47360,72140,119920,155050,207570
4,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,11-2030,Public Relations and Fundraising Managers,detailed,40,29.7,0.571,1.06,*,*,*,*,*,*,*,*,*,*,*,*,*


Did this exploration after dropping NaN

In [8]:
bls_m2019.describe(exclude='number')

,area_title,naics_title,i_group,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90
count,142410,142410,142410,142410,142410,142410,142410,142410,142410,142410,142410,142410,142410,142410,142410,142410,142410,142410,142410,142410,142410,142410,142410
unique,396,1,1,806,806,3,3903,501,21778,1571,7925,13977,297,4916,5883,6768,7833,8738,8623,10460,12425,14786,16865
top,"New York-Newark-Jersey City, NY-NJ-PA",Cross-industry,cross-industry,53-0000,"Secretaries and Administrative Assistants, Exc...",detailed,**,**,**,**,*,*,3,*,*,*,*,*,*,*,*,*,#
freq,759,142410,142410,396,396,133369,11719,11719,11719,11719,9221,2652,3008,9221,9221,9221,9221,9221,2652,2652,2652,2652,3977


In [9]:
bls_m2019.describe()

,area,area_type,naics,own_code
count,142410.000000,142410.0,142410.0,142410.0
mean,32528.190506,4.0,0.0,1235.0
std,15255.350647,0.0,0.0,0.0
min,10180.000000,4.0,0.0,1235.0
25%,20220.000000,4.0,0.0,1235.0
50%,31080.000000,4.0,0.0,1235.0
75%,41620.000000,4.0,0.0,1235.0
max,79600.000000,4.0,0.0,1235.0


In [10]:
bls_m2019['area_title'].nunique()

396

In [11]:
bls_m2019.sort_values(['area_title', 'loc_quotient'], ascending=False)

,area,area_title,area_type,naics,naics_title,i_group,own_code,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90
133989,49740,"Yuma, AZ",4,0,Cross-industry,cross-industry,1235,11-9021,Construction Managers,detailed,**,**,**,**,49.09,102110,4.4,22.77,30.47,47.28,68.88,77.95,47370,63370,98340,143280,162140
134049,49740,"Yuma, AZ",4,0,Cross-industry,cross-industry,1235,25-3031,"Substitute Teachers, Short-Term",detailed,**,**,**,**,14.27,29680,2.8,11.84,12.64,13.96,15.99,17.91,24640,26280,29040,33260,37250
134065,49740,"Yuma, AZ",4,0,Cross-industry,cross-industry,1235,29-1131,Veterinarians,detailed,**,**,**,**,55.96,116400,14.6,15.27,36.96,58.8,73.94,84.62,31750,76870,122310,153800,176010
134069,49740,"Yuma, AZ",4,0,Cross-industry,cross-industry,1235,29-1216,General Internal Medicine Physicians,detailed,**,**,**,**,103.73,215750,12.8,53.13,62.79,#,#,#,110520,130600,#,#,#
134071,49740,"Yuma, AZ",4,0,Cross-industry,cross-industry,1235,29-1292,Dental Hygienists,detailed,**,**,**,**,38.22,79500,9.4,27.98,31.68,40.21,44.89,47.68,58190,65890,83650,93370,99180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,13-1111,Management Analysts,detailed,80,30.3,1.15,0.24,38.36,79780,3.1,24.83,28.65,36.46,45.76,59.26,51650,59600,75830,95190,123270
37,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,15-1211,Computer Systems Analysts,detailed,60,23.7,0.917,0.23,31.14,64770,4.5,20.39,22.77,28.77,39.04,46.91,42410,47350,59840,81200,97580
17,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,11-9198,"Personal Service Managers, All Other; Entertai...",detailed,30,19.9,0.523,0.16,45.95,95580,8.6,18.91,32.23,41.9,50.89,73.42,39330,67050,87160,105850,152710
42,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,15-1256,Software Developers and Software Quality Assur...,detailed,90,21.1,1.368,0.14,42.77,88960,6.2,24.29,30.02,40.85,52.61,66.44,50520,62440,84980,109430,138190


In [12]:
# Clean up loc_quotient
bls_m2019['loc_quotient'] = bls_m2019['loc_quotient'].replace('**', '0')
bls_m2019['loc_quotient'] = bls_m2019['loc_quotient'].astype(float)

In [13]:
# Clean up jobs_1000
bls_m2019["jobs_1000"] = bls_m2019['jobs_1000'].replace("**","0")
bls_m2019["jobs_1000"] = bls_m2019['jobs_1000'].astype(float)

In [14]:
# Clean up h_mean
# replacing *, #'s with 0's for now -> will calculate after cleaning
bls_m2019['h_mean'] = bls_m2019['h_mean'].replace("*","0")
bls_m2019['h_mean'] = bls_m2019['h_mean'].replace("#","0")
bls_m2019['h_mean'] = bls_m2019['h_mean'].astype(float)

In [15]:
# Clean up a_mean
bls_m2019['a_mean'] = bls_m2019['a_mean'].replace("*","0")
bls_m2019['a_mean'] = bls_m2019['a_mean'].replace("#","0")
bls_m2019['a_mean'] = bls_m2019['a_mean'].astype(int)

## Feature Engineering


In [16]:
# We do not need all the columns so I will compress the df to just the necessary columns
bls = bls_m2019[['area_title', 'occ_title', 'jobs_1000', 'loc_quotient', 'h_mean', 'a_mean']]

In [17]:
# For readability
bls = bls.rename(columns = {
    'h_mean': 'avg_hourly_salary',
    'a_mean': 'avg_annual_salary'
})

In [18]:
# Creating a city and state columns for easier manipulation and joining with other tables
states = []
cities = []

for row in bls_m2019['area_title']:
  row = row.split(',')
  cities.append(row[0])
  states.append(row[1])

bls['city'] = cities
bls['state'] = states

In [19]:
print(bls.shape)
bls.head()

(142410, 8)


,area_title,occ_title,jobs_1000,loc_quotient,avg_hourly_salary,avg_annual_salary,city,state
0,"Abilene, TX",All Occupations,1000.000,1.00,20.08,41760,Abilene,TX
1,"Abilene, TX",Management Occupations,40.137,0.73,43.51,90500,Abilene,TX
2,"Abilene, TX",General and Operations Managers,16.499,1.01,40.39,84000,Abilene,TX
3,"Abilene, TX",Sales Managers,1.728,0.63,59.96,124710,Abilene,TX
4,"Abilene, TX",Public Relations and Fundraising Managers,0.571,1.06,0.00,0,Abilene,TX


In [20]:
# Checking column types after revision
bls.dtypes

area_title            object
occ_title             object
jobs_1000            float64
loc_quotient         float64
avg_hourly_salary    float64
avg_annual_salary      int64
city                  object
state                 object
dtype: object

In [21]:
bls.isnull().sum()

area_title           0
occ_title            0
jobs_1000            0
loc_quotient         0
avg_hourly_salary    0
avg_annual_salary    0
city                 0
state                0
dtype: int64

In [22]:
# Reorder df, also do not need area_tile (redundant)
bls = bls[['state', 'city', 'occ_title', 'jobs_1000', 'loc_quotient', 'avg_hourly_salary', 'avg_annual_salary']]

In [23]:
bls.loc[(bls.city =='Urban Honolulu'), 'city'] = 'Honolulu'

In [24]:
bls.loc[(bls.city == 'Honolulu')][0:1]

,state,city,occ_title,jobs_1000,loc_quotient,avg_hourly_salary,avg_annual_salary
124580,HI,Honolulu,All Occupations,1000.0,1.0,27.43,57060


### Calculate Annual Wage
- using hourly wage 
(assumption - 40 hours * 52 weeks)

In [25]:
bls['avg_annual_salary'] = np.where(bls['avg_annual_salary']== 0, bls['avg_hourly_salary']*40*52, bls['avg_annual_salary'] )

#### Check if columns to see if formula is applied properly

In [26]:
14.07*40*52

29265.6

In [27]:
assert bls['avg_annual_salary'][1290] == 29265.6

In [28]:
bls.loc[
        (bls.city == 'Albany')
        & (bls.occ_title == 'Musicians and Singers')]


,state,city,occ_title,jobs_1000,loc_quotient,avg_hourly_salary,avg_annual_salary
1290,OR,Albany,Musicians and Singers,1.589,5.67,14.07,29265.6


In [29]:
20.97*40*52

43617.6

In [30]:
assert bls['avg_annual_salary'][1691] == 43617.6

In [31]:

bls.loc[
        (bls.city == 'Albany-Schenectady-Troy')
        & (bls.occ_title == 'Miscellaneous Entertainers and Performers, Sports and Related Workers')]

,state,city,occ_title,jobs_1000,loc_quotient,avg_hourly_salary,avg_annual_salary
1691,NY,Albany-Schenectady-Troy,"Miscellaneous Entertainers and Performers, Spo...",0.138,1.28,20.97,43617.6


In [32]:
26.23*40*52

54558.4

In [33]:
assert bls['avg_annual_salary'][8367] == 54558.4

In [34]:
bls.loc[
        (bls.city == 'Austin-Round Rock')
        & (bls.occ_title == 'Musicians and Singers')]

,state,city,occ_title,jobs_1000,loc_quotient,avg_hourly_salary,avg_annual_salary
8367,TX,Austin-Round Rock,Musicians and Singers,0.374,1.33,26.23,54558.4


In [35]:
bls.loc[
        (bls.city == 'Denver-Aurora-Lakewood')
        & (bls.occ_title == 'Actors')]

,state,city,occ_title,jobs_1000,loc_quotient,avg_hourly_salary,avg_annual_salary
33443,CO,Denver-Aurora-Lakewood,Actors,0.33,0.92,33.1,68848.0


## Calculate Hourly Wage
- use annual wage(assumption - /40 and /52)

In [36]:
bls['avg_hourly_salary'] = np.where(bls['avg_hourly_salary']== 0, bls['avg_annual_salary']/(40*52), bls['avg_hourly_salary'] )

#### Check if columns to see if formula is applied properly

In [37]:
17400.0/(40*52)

8.365384615384615

In [38]:
assert bls['avg_hourly_salary'][103837] == 8.365384615384615

In [39]:
bls.loc[
        (bls.city == 'St. George')
        & (bls.occ_title == 'Legislators')]

,state,city,occ_title,jobs_1000,loc_quotient,avg_hourly_salary,avg_annual_salary
103837,UT,St. George,Legislators,0.502,1.41,8.365385,17400.0


In [40]:
51870.0/(40*52)

24.9375

In [41]:
assert bls['avg_hourly_salary'][21409]==24.9375

In [42]:
bls.loc[
        (bls.city == 'Charlotte-Concord-Gastonia')
        & (bls.occ_title == 'Special Education Teachers, Kindergarten and Elementary School')]

,state,city,occ_title,jobs_1000,loc_quotient,avg_hourly_salary,avg_annual_salary
21409,NC-SC,Charlotte-Concord-Gastonia,"Special Education Teachers, Kindergarten and E...",1.337,1.01,24.9375,51870.0


In [43]:
92190/(40*52)

44.32211538461539

In [44]:
assert bls['avg_hourly_salary'][17786]==44.32211538461539

In [45]:
bls.loc[
        (bls.city == 'Canton-Massillon')
        & (bls.occ_title == 'Chemistry Teachers, Postsecondary')]

,state,city,occ_title,jobs_1000,loc_quotient,avg_hourly_salary,avg_annual_salary
17786,OH,Canton-Massillon,"Chemistry Teachers, Postsecondary",0.18,1.24,44.322115,92190.0


## Save modified df 

In [46]:
bls.to_csv('bls.csv')

In [49]:
!cp bls.csv "drive/My Drive/"